In [1]:
import pandas as pd
import numpy as np
import metapredict as meta

In [3]:
# REPLACE WITH YOUR CSV
df = pd.read_csv("ZincFinger_Classical_RBD.csv")

C:\Users\faris\AppData\Local\Temp\ipykernel_31620\4156281306.py:2: DtypeWarning: Columns (11,12,13,1134,1138,1146,1154,1158,1162,1166,1170,1174,1178,1182,1186,1190,1194,1206,1214,1218,1222,1226,1230,1234,1238,1242,1250,1254,1258,1262,1266,1270,1274,1278,1282,1286,1290,1294,1298,1302,1306,1310,1314,1318,1332,1336,1340,1348,1352,1356,1360,1364,1376,1380,1384,1392,1408,1412,1428) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("ZincFinger_Classical_RBD.csv")


In [4]:
new_cols = ["Meta_disordered_index", "Meta_disordered_position", "Meta_disordered_domains", "Meta_disordered_sequence", "Meta_folded_index", "Meta_folded_position", "Meta_folded_domains", "Meta_folded_sequence", "Meta_disordered_content_fraction", "Meta_disordered_content_count", "Meta_folded_content_fraction", "Meta_folded_content_count", "Meta_number_disordered_regions", "Meta_average_disordered_region_size", "Meta_number_folded_regions", "Meta_average_folded_region_size"]

In [5]:
sequences = list(df["sequence"])

In [ ]:
disorder_objects = meta.predict_disorder(sequences, version = "V3", return_domains = True, disorder_threshold = 0.5)

In [ ]:
disordered_domain_boundaries = [obj.disordered_domain_boundaries for obj in disorder_objects]
folded_domain_boundaries = [obj.folded_domain_boundaries for obj in disorder_objects]
disordered_domains = [obj.disordered_domains for obj in disorder_objects]
folded_domains = [obj.folded_domains for obj in disorder_objects]

In [ ]:
def format_boundaries(boundary_list):
    return ",".join([f"{start+1}..{end}" for start, end in boundary_list])

In [ ]:
def join_domains(domain_list):
    return "..".join(domain_list)

In [ ]:
disordered_domain_positions = [format_boundaries(obj) for obj in disordered_domain_boundaries]
folded_domain_positions = [format_boundaries(obj) for obj in folded_domain_boundaries]
disordered_sequence = [join_domains(obj) for obj in disordered_domains]
folded_sequence = [join_domains(obj) for obj in folded_domains]

In [ ]:
cols = [disordered_domain_boundaries, disordered_domain_positions, disordered_domains, disordered_sequence, folded_domain_boundaries, folded_domain_positions, folded_domains, folded_sequence]

In [ ]:
def get_seq_length(seq):
    return len(seq) if pd.notna(seq) else np.nan

In [ ]:
# Sequence length
seq_lengths = rbp["sequence"].apply(get_seq_length)

# Region counts
num_disordered = [len(obj.disordered_domain_boundaries) for obj in disorder_objects]
num_folded = [len(obj.folded_domain_boundaries) for obj in disorder_objects]

# Region sizes
avg_disordered_size = [
    np.mean([end - start for start, end in obj.disordered_domain_boundaries]) if obj.disordered_domain_boundaries else 0
    for obj in disorder_objects
]
avg_folded_size = [
    np.mean([end - start for start, end in obj.folded_domain_boundaries]) if obj.folded_domain_boundaries else 0
    for obj in disorder_objects
]

# Content counts
disordered_counts = [
    sum([end - start for start, end in obj.disordered_domain_boundaries]) for obj in disorder_objects
]
folded_counts = [
    sum([end - start for start, end in obj.folded_domain_boundaries]) for obj in disorder_objects
]

# Fractions
disordered_fractions = [count / length if length else np.nan for count, length in zip(disordered_counts, seq_lengths)]
folded_fractions = [count / length if length else np.nan for count, length in zip(folded_counts, seq_lengths)]


In [ ]:
more_cols = [disordered_fractions, disordered_counts, folded_fractions, folded_counts, num_disordered, avg_disordered_size, num_folded, avg_folded_size]

In [ ]:
cols_to_add = cols + more_cols

In [ ]:
for name, values in zip(new_cols, cols_to_add):
    df[name] = values